#### [ML - 생성 분류 모델 만들기 3]

- 주제: 2개 종류 생선 분류 모델 만들기
- 데이터: ../Data/fish.csv
- 피쳐들: 길이, 무게 .실수형
- 타겟: 생선의 종류
- 학습종류: 지도학습 -> 분류
- 학습방법: 다양한 분류 알고리즘들 -> KNN 최근접이웃알고리즘

[1] 데이터 확인 : 피쳐와 타겟 <hr>

In [12]:
# [1-1] 모듈 로딩 및 데이터 준비
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib

# ML학습 알고리즘 모델
from sklearn.neighbors import KNeighborsClassifier

In [13]:
# CoW 기능 활성화 : 데이터 변경시 복사 후 처리
if not pd.get_option('mode.copy_on_write'):  
    pd.options.mode.copy_on_write = True   
    print(f"CoW: {pd.get_option('mode.copy_on_write')}")  

In [ ]:
# [1-2] 데이터 로딩 및 빙어와 도미 데이터만 추출

DATA_FILE = '../Data/fish.csv'

dataDF = pd.read_csv(DATA_FILE, usecols=[0, 1, 2])
display(dataDF.head())
dataDF['Species'].unique()

# bream 도미, smelt 빙어 행만 추출
fishDF = dataDF[(dataDF.Species == 'Bream') | (dataDF.Species == 'Smelt')]

# 행인덱스 초기화
fishDF.reset_index(drop=True, inplace=True)
display(fishDF.head(), fishDF.tail(3))

,Species,Weight,Length
0,Bream,242.0,25.4
1,Bream,290.0,26.3
2,Bream,340.0,26.5
3,Bream,363.0,29.0
4,Bream,430.0,29.0


,Species,Weight,Length
0,Bream,242.0,25.4
1,Bream,290.0,26.3
2,Bream,340.0,26.5
3,Bream,363.0,29.0
4,Bream,430.0,29.0


,Species,Weight,Length
46,Smelt,12.2,13.0
47,Smelt,19.7,14.3
48,Smelt,19.9,15.0


   Species  Weight  Length
0    Bream   242.0    25.4
1    Bream   290.0    26.3
2    Bream   340.0    26.5
3    Bream   363.0    29.0
4    Bream   430.0    29.0
5    Bream   450.0    29.7
6    Bream   500.0    29.7
7    Bream   390.0    30.0
8    Bream   450.0    30.0
9    Bream   500.0    30.7
10   Bream   475.0    31.0
11   Bream   500.0    31.0
12   Bream   500.0    31.5
13   Bream   340.0    32.0
14   Bream   600.0    32.0
15   Bream   600.0    32.0
16   Bream   700.0    33.0
17   Bream   700.0    33.0
18   Bream   610.0    33.5
19   Bream   650.0    33.5
20   Bream   575.0    34.0
21   Bream   685.0    34.0
22   Bream   620.0    34.5
23   Bream   680.0    35.0
24   Bream   700.0    35.0
25   Bream   725.0    35.0
26   Bream   720.0    35.0
27   Bream   714.0    36.0
28   Bream   850.0    36.0
29   Bream  1000.0    37.0
30   Bream   920.0    38.5
31   Bream   955.0    38.5
32   Bream   925.0    39.5
33   Bream   975.0    41.0
34   Bream   950.0    41.0
35   Smelt     6.7     9.8
3

In [15]:
# [1-3] 
# 품종 컬럼 타입 변환 저장
speciesSR = fishDF.Species
fishDF.Species = fishDF.Species.astype('category')

In [16]:
# 전체 데이터 타입 확인
fishDF.info()

# 카테고리 접근자 cat 사용
print(f"[categories] => {fishDF.Species.cat.categories}")
print(f"[categories] => \n{fishDF.Species.cat.codes}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   Species  49 non-null     category
 1   Weight   49 non-null     float64 
 2   Length   49 non-null     float64 
dtypes: category(1), float64(2)
memory usage: 1.1 KB
[categories] => Index(['Bream', 'Smelt'], dtype='object')
[categories] => 
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    1
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    1
47    1
48    1
dtype: int8


[2] 피쳐/데이터와 타겟/라벨 분리 <hr>

In [17]:
# 학습용 피쳐/데이터와 타겟/라벨 분리
# -> 피쳐/데이터 : 2차원
featureDF = fishDF[fishDF.columns[1:]]

# -> 타겟/라벨 : 1차원
targetSR = fishDF[fishDF.columns[0]]

print(f"featureDF : {featureDF.shape} / {featureDF.ndim}D, targetSR : {targetSR.shape} / {targetSR.ndim}D ")

featureDF : (49, 2) / 2D, targetSR : (49,) / 1D 


[3] 학습 진행 <hr>

In [18]:
# 모델 인스턴스 생성
kModel = KNeighborsClassifier()
kModel

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [19]:
# 학습 진행 : 학습용데이터(2D)와 타겟(1D)
# KNN 학습 알고리즘은 메모리에 데이터와 타겟을 로딩만 지행
kModel.fit(featureDF, targetSR)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


[4] 평가진행 : 테스트용 데이터/피쳐와 타겟 <hr>
- 학습용 데이터 평가결과 테스트용 데이터 평가결과 비교

In [20]:
kModel.score(featureDF, targetSR)

1.0

[5] 모델 활용 서비스 <hr>

In [21]:
# 아무 도미 또는 빙어의 길이/무게 ==> 도미와 빙어 예측
in_data = input("생선의 무게, 길이 입력(예: 120 23) : ").strip().split()
# in_data = [float(data) for data  in in_data]
in_data = list(map(float, in_data))

new_data = pd.DataFrame([in_data], columns=fishDF.columns[1:])

print(kModel.predict(new_data))

ValueError: 2 columns passed, passed data had 0 columns